In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [4]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# Monitering the LLM
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning mistral Instruct 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kingabzpro. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231025_165130-qpfs292m
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run visionary-shadow-4
wandb: ⭐️ View project at https://wandb.ai/kingabzpro/Fine%20tuning%20mistral%20Instruct%207B
wandb: 🚀 View run at https://wandb.ai/kingabzpro/Fine%20tuning%20mistral%20Instruct%207B/runs/qpfs292m


In [6]:
base_model = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b-instruct-guanaco"

In [7]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train")
dataset["text"][100]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/mlabonne--guanaco-llama2-1k-f1f1134768f90029/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


'<s>[INST] cuanto es 2x2 xD [/INST] La respuesta es 4. </s><s>[INST] puedes demostrarme matematicamente que 2x2 es 4? [/INST] En una multiplicación, el producto es el resultado de sumar un factor tantas veces como indique el otro, es decir, si tenemos una operación v · n = x, entonces x será igual a v sumado n veces o n sumado v veces, por ejemplo, para la multiplicación 3 · 4 podemos sumar "3 + 3 + 3 + 3" o "4 + 4 + 4" y en ambos casos nos daría como resultado 12, para el caso de 2 · 2 al ser iguales los dos factores el producto sería "2 + 2" que es igual a 4 </s>'

In [8]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(True, True)

In [9]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [10]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)


In [11]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:173: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.246700
50,1.614200
75,1.198900
100,1.397300
125,1.156800
150,1.326100
175,1.171000
200,1.428400
225,1.135400
250,1.479600


TrainOutput(global_step=250, training_loss=1.3154491577148437, metrics={'train_runtime': 2853.1005, 'train_samples_per_second': 0.35, 'train_steps_per_second': 0.088, 'total_flos': 1.874641569231667e+16, 'train_loss': 1.3154491577148437, 'epoch': 1.0})

In [13]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                    train/epoch ▁▂▃▃▄▅▆▆▇██
wandb:              train/global_step ▁▂▃▃▄▅▆▆▇██
wandb:            train/learning_rate ▁▁▁▁▁▁▁▁▁▁
wandb:                     train/loss ▃█▂▅▁▄▂▅▁▆
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                    train/epoch 1.0
wandb:              train/global_step 250
wandb:            train/learning_rate 0.0002
wandb:                     train/loss 1.4796
wandb:               train/total_flos 1.874641569231667e+16
wandb:               train/train_loss 1.31545
wandb:            train/train_runtime 2853.1005
wandb: train/train_samples_per_second 0.35
wandb:   train/train_steps_per_second 0.088
wandb: 
wandb: 🚀 View run visionary-shadow-4 at: https://wandb.ai/kingabzpro

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=1024, bias=False
      

In [14]:
trainer.model.push_to_hub(new_model, use_temp_dir=False)

adapter_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kingabzpro/mistral_7b-instruct-guanaco/commit/d0af8f96747ebbc90be2179eceedda4729df02fd', commit_message='Upload model', commit_description='', oid='d0af8f96747ebbc90be2179eceedda4729df02fd', pr_url=None, pr_revision=None, pr_num=None)

In [15]:
logging.set_verbosity(logging.CRITICAL)

prompt = "How do I find true love?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I find true love? [/INST] Finding true love is a complex and personal journey, and there is no one-size-fits-all answer to this question. However, here are some general tips that may help you on your journey:

1. Be yourself: The most important thing is to be true to yourself and to be comfortable in your own skin. When you are confident and happy with who you are, you are more likely to attract the right person.

2. Set boundaries: It's important to set boundaries and to be clear about what you are looking for in a partner. This will help you to avoid wasting time on people who are not a good match for you.

3. Be open-minded: Don't limit yourself to a certain type of person or to a certain place. Be open to new experiences and to meeting people from different backgrounds.

4. Be patient:


In [16]:
prompt = "What is Datacamp Career track?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is Datacamp Career track? [/INST] Datacamp Career Track is a program that provides a comprehensive learning path for individuals who want to become data scientists. It covers a wide range of topics, including data analysis, machine learning, and data visualization. The program is designed to help individuals develop the skills and knowledge needed to succeed in the field of data science. 
Datacamp Career Track is a self-paced program that allows individuals to learn at their own pace. It includes a variety of interactive exercises, quizzes, and projects that help individuals apply their knowledge and skills. The program also includes a community of other data scientists who can provide support and feedback. 
Overall, Datacamp Career Track is a great option for individuals who want to become data scientists and need a structured learning path to achieve their goals. 
Is there anything else you would like to know about Datacamp Career Track? 
Please let
